In [1]:
! pip install transformers
# sentencepiece library for t5 transformers
! pip install sentencepiece
! pip install sumy

     |████████████████████████████████| 2.9 MB 11.2 MB/s 
     |████████████████████████████████| 636 kB 31.5 MB/s 
     |████████████████████████████████| 56 kB 2.7 MB/s 
     |████████████████████████████████| 3.3 MB 31.8 MB/s 
     |████████████████████████████████| 895 kB 49.4 MB/s 
     |████████████████████████████████| 109 kB 54.2 MB/s 
     |████████████████████████████████| 546 kB 43.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 12.4 MB/s 
     |████████████████████████████████| 83 kB 2.1 MB/s 
     |████████████████████████████████| 10.1 MB 26.3 MB/s 
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21711 sha256=366c5e17ec6d7e4753a73cc87ce77bc2eb450e534f97dc023fbb2e08bcd66574
  Stored in directory: /root/.cache/pip/wheels/d4/bf/51/81d27ad638e1a6dca4f362ecc33d1e2c764b8ea7ec751b8fc1


## Class Text Summarizer:


>  Class return the text summarisation performed on given text  using diffferent different state-of-the-art text summmarizer



In [3]:
import torch
import sumy
import gensim
from gensim.summarization import summarize
import nltk
nltk.download('punkt')
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig,BartModel
from transformers import GPT2Tokenizer,GPT2LMHeadModel
from transformers import XLMWithLMHeadModel, XLMTokenizer
from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration
# Import the LexRank summarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
# Importing the parser and tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
# Import the LexRank summarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
lex_rank_summarizer = LexRankSummarizer() 
from sumy.summarizers.lsa import LsaSummarizer
lsa_summarizer=LsaSummarizer()
# Parsing the text string using PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser



class TextSummarizer:
  def __init__(self):
    # Instantiating the model and tokenizer with gpt-2 and bart and XLM
    self.tokenizer_bart_large = BartTokenizer.from_pretrained('facebook/bart-large')
    self.model_bart_large = BartModel.from_pretrained('facebook/bart-large')
    self.tokenizer_gpt2=GPT2Tokenizer.from_pretrained('gpt2')
    self.model_gpt2=GPT2LMHeadModel.from_pretrained('gpt2')
    self.tokenizer_bart=BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    self.model_bart=BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
    # Instantiating the model and tokenizer xlm 
    self.tokenizer_XLM=XLMTokenizer.from_pretrained('xlm-mlm-en-2048')
    self.model_XLM=XLMWithLMHeadModel.from_pretrained('xlm-mlm-en-2048')
    self.model_t5 = T5ForConditionalGeneration.from_pretrained('t5-small')
    self.tokenizer_t5 = T5Tokenizer.from_pretrained('t5-small')
    self.tokenizer_list = [self.tokenizer_gpt2,self.tokenizer_bart, self.tokenizer_XLM]
    self.model_list = [self.model_gpt2,self.model_bart, self.model_XLM]

  def text_summarizer(self, text):
    text_summary_list =[]
    for models in range(len(self.model_list)):
      print('summarization using ' ,  self.model_list[models].name_or_path)
      
      if  self.model_list[models].name_or_path == "xlm-mlm-en-2048":
        # print('true')
        text = text[0:2500]

      inputs=self.tokenizer_list[models](text, return_tensors='pt' )
      summary_ids=self.model_list[models].generate(inputs['input_ids'], early_stopping=True, min_length = 15, max_length= 20)
      text_summary= self.tokenizer_list[models].batch_decode(summary_ids)
      text_summary_list.append(text_summary)
      print(text_summary)
    # Concatenating the word "summarize:" to raw text
    t5_text = "summarize:" + text
    # encoding the input text
    input_ids=self.tokenizer_t5.encode(t5_text, return_tensors='pt', truncation=True)
    summary_ids = self.model_t5.generate(input_ids,early_stopping=True, min_length = 15, max_length= 20)
    t5_summary = self.tokenizer_t5.decode(summary_ids[0], skip_special_tokens=True)
    text_summary_list.append(t5_summary)
    summary_genensim = summarize(text)
    text_summary_list.append(summary_genensim)
    my_parser = PlaintextParser.from_string(text,Tokenizer('english'))
    lexrank_summary_sentences = lex_rank_summarizer(my_parser.document,sentences_count=3)
    lexrank_summary = ""
    for sentence in lexrank_summary_sentences:
      lexrank_summary= lexrank_summary + " "+ str(sentence)
    text_summary_list.append(lexrank_summary)

    # creating the lsa summarizer

    parser=PlaintextParser.from_string(text,Tokenizer('english'))
    lsa_summary_sentences= lsa_summarizer(parser.document,3)
    lsa_summary =""
    # Printing the summary
    for sentence in lsa_summary_sentences:
        lsa_summary= lsa_summary+" "+ str(sentence)
    text_summary_list.append(lsa_summary)
    
    return text_summary_list



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
textSummarizer  = TextSummarizer()   
text = """Junk foods taste good that’s why it is mostly liked by everyone of any age group especially kids and school going children. They generally ask for the junk food daily because they have been trend so by their parents from the childhood. They never have been discussed by their parents about the harmful effects of junk foods over health. According to the research by scientists, it has been found that junk foods have negative effects on the health in many ways. They are generally fried food found in the market in the packets. They become high in calories, high in cholesterol, low in healthy nutrients, high in sodium mineral, high in sugar, starch, unhealthy fat, lack of protein and lack of dietary fibers. Processed and junk foods are the means of rapid and unhealthy weight gain and negatively impact the whole body throughout the life. It makes able a person to gain excessive weight which is called as obesity. Junk foods tastes good and looks good however do not fulfil the healthy calorie requirement of the body. Some of the foods like french fries, fried foods, pizza, burgers, candy, soft drinks, baked goods, ice cream, cookies, etc are the example of high-sugar and high-fat containing foods. It is found according to the Centres for Disease Control and Prevention that Kids and children eating junk food are more prone to the type-2 diabetes. In type-2 diabetes our body become unable to regulate blood sugar level
  """
  

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

...

Some weights of XLMWithLMHeadModel were not initialized from the model checkpoint at xlm-mlm-en-2048 and are newly initialized: ['transformer.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [5]:
text_summary_list = textSummarizer.text_summarizer(text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 290, but ``max_length`` is set to 20.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


summarization using  gpt2
['Junk foods taste good that’s why it is mostly liked by everyone of any age group especially kids and school going children. They generally ask for the junk food daily because they have been trend so by their parents from the childhood. They never have been discussed by their parents about the harmful effects of junk foods over health. According to the research by scientists, it has been found that junk foods have negative effects on the health in many ways. They are generally fried food found in the market in the packets. They become high in calories, high in cholesterol, low in healthy nutrients, high in sodium mineral, high in sugar, starch, unhealthy fat, lack of protein and lack of dietary fibers. Processed and junk foods are the means of rapid and unhealthy weight gain and negatively impact the whole body throughout the life. It makes able a person to gain excessive weight which is called as obesity. Junk foods tastes good and looks good however do not 

Input length of input_ids is 297, but ``max_length`` is set to 20.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


["<s>junk foods taste good that's why it is mostly liked by everyone of any age group especially kids and school going children. they generally ask for the junk food daily because they have been trend so by their parents from the childhood. they never have been discussed by their parents about the harmful effects of junk foods over health. according to the research by scientists, it has been found that junk foods have negative effects on the health in many ways. they are generally fried food found in the market in the packets. they become high in calories, high in cholesterol, low in healthy nutrients, high in sodium mineral, high in sugar, starch, unhealthy fat, lack of protein and lack of dietary fibers. processed and junk foods are the means of rapid and unhealthy weight gain and negatively impact the whole body throughout the life. it makes able a person to gain excessive weight which is called as obesity. junk foods tastes good and looks good however do not fulfil the healthy calo

In [6]:
text_summary_list

[['Junk foods taste good that’s why it is mostly liked by everyone of any age group especially kids and school going children. They generally ask for the junk food daily because they have been trend so by their parents from the childhood. They never have been discussed by their parents about the harmful effects of junk foods over health. According to the research by scientists, it has been found that junk foods have negative effects on the health in many ways. They are generally fried food found in the market in the packets. They become high in calories, high in cholesterol, low in healthy nutrients, high in sodium mineral, high in sugar, starch, unhealthy fat, lack of protein and lack of dietary fibers. Processed and junk foods are the means of rapid and unhealthy weight gain and negatively impact the whole body throughout the life. It makes able a person to gain excessive weight which is called as obesity. Junk foods tastes good and looks good however do not fulfil the healthy calori